In [1]:
!pip install -q aiohttp
import asyncio
import requests
import aiohttp

#https://www.blog.pythonlibrary.org/2016/11/09/an-intro-to-aiohttp/

     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 317kB 45.1MB/s 
     |████████████████████████████████| 256kB 45.7MB/s 


In [0]:
URL = 'https://xkcd.com/{num}/info.0.json'

MAX_POST = 700


In [0]:
def get_urls():
  posts = []
  for post in range(MAX_POST):
    response = requests.get(URL.format(num=post))
    posts.append(posts)
  return posts

In [0]:
%timeit get_urls()

1 loop, best of 3: 48.2 s per loop


In [0]:
async def get_url_async(url):
  print('Getting url',url)
#   response = 
  print('Obtained url',url)
  return await requests.get(url)

In [0]:
async def get_urls_async(num_items):
  print('Building futures')
  future_pages = [  get_url_async(URL.format(num=i)) 
             for i in range(num_items) ]
  print('Waiting to complete jobs')
  complete, pending = await asyncio.wait(future_pages)
  pages = [ f.result() for f in complete ]
  
  return pages

In [0]:
def get_async(num_items):
  loop = asyncio.new_event_loop()
  try:
    print('running tasks ...')
    loop.run_until_complete(get_urls_async(num_items))
    print('Tasks ended')
#     return urls
  finally:
    print('Closing')
    loop.close()

In [0]:
get_async(1)

running tasks ...
Building futures
Waiting to complete jobs
Getting url https://xkcd.com/0/info.0.json
Obtained url https://xkcd.com/0/info.0.json
Closing


TypeError: ignored

In [0]:
async def fetch(session,url):
  async with session.get(url) as response:
    return await response.text()

async def main(num_pages):

  async with aiohttp.ClientSession() as session:
    future_pages = [ fetch(session,URL.format(num=i)) for i in range(num_pages)]

#     html = await fetch(session, 'http://python.org' )
    complete, pending = await asyncio.wait(future_pages)
    pages = [t.result() for t in complete ]
    return pages

def get_aio(num_pages):
  loop = asyncio.new_event_loop()
  content = loop.run_until_complete(main(num_pages))
  return content

In [0]:
# %timeit get_aio(MAX_POST)
get_aio(MAX_POST)

700

In [0]:
from concurrent.futures import ThreadPoolExecutor

def get_thread():
    with ThreadPoolExecutor() as ex:
      threads = []
      posts = []
      for post in range(MAX_POST):
        t = ex.submit(requests.get,URL.format(num=post))
        threads.append(t)

      try:
        for t in threads:
          data = t.result()
          posts.append(data.content)
      except:
        raise
    return posts

In [0]:
%timeit get_thread()

1 loop, best of 3: 8.65 s per loop
